In [1]:
from scrape_school_url import *
import pandas as pd
from bs4 import BeautifulSoup
import re
from os import path, makedirs

In [2]:
sdi = SchoolDataIndex()

In [3]:
school_ids = list(sdi.school_ids())

In [4]:
interested_urls = {key: re.sub('^.*/', '', val) for key, val in SCRAPING_URLS.items()}

def prep_param_dict(soup):
  param_dict = {}
  for anchor in soup.find_all('a'):
    topic = [kurl for kurl, iurl in interested_urls.items() if iurl in anchor.attrs['href']]
    if topic:
      param_dict[topic[0]] = re.sub('^.*\?', '', anchor.attrs['href'])
  return param_dict

In [19]:
sd = load_school_data(school_ids[490], only=['general'])
soup = load_soup(sd['general'])
params_dict = prep_param_dict(soup)
sd = load_school_data('1020080005', params_dict, force=True)
sd

{'general': 'html/2022-09/general/1020080005.html',
 'student': 'html/2022-09/student/1020080005.html',
 'staff': 'html/2022-09/staff/1020080005.html',
 'computer_internet': 'html/2022-09/computer_internet/1020080005.html',
 'building': 'html/2022-09/building/1020080005.html',
 'durable_goods': 'html/2022-09/durable_goods/1020080005.html'}

In [6]:
trim_re = r'[\:\s]*$'

In [7]:
about_school = []
def _clean_text(string: str) -> str:
  string = string.strip()
  string = re.sub(trim_re, '', string)
  string = re.sub('\s+',' ', string)
  return string

table = soup.find('table').find('table', attrs={'width': '521'})
for table_row in table.find_all('tr'):
  cells = table_row.find_all('td')

  if len(cells) == 2:
    key_name = _clean_text(cells[0].text)
    value = _clean_text(cells[1].text)

    if cells[1].find_all('a'):
      value = [{
        'text': a_tag.text,
        'href': a_tag.attrs['href']}
        for a_tag in cells[1].find_all('a') if a_tag.text]
    
    about_school.append({
        'key': key_name,
        'value': value
      })
  else:
    about_school.append({'value': _clean_text(cells[0].text)})
about_school

[{'key': 'รหัสโรงเรียน 10 หลัก', 'value': '1020080005'},
 {'key': 'รหัส Smis 8 หลัก', 'value': '20010029'},
 {'key': 'รหัส Obec 6 หลัก', 'value': '080005'},
 {'key': 'ชื่อสถานศึกษา(ไทย)', 'value': 'วัดศรีพโลทัย'},
 {'key': 'ชื่อสถานศึกษา(อังกฤษ)', 'value': 'WATSRIPALOTHAI'},
 {'key': 'ที่อยู่', 'value': 'หมู่ที่ 1 บ้านหนองไม้แดง'},
 {'key': 'ตำบล', 'value': 'หนองไม้แดง'},
 {'key': 'อำเภอ', 'value': 'เมืองชลบุรี'},
 {'key': 'จังหวัด', 'value': 'ชลบุรี'},
 {'key': 'รหัสไปรษณีย์', 'value': '20000'},
 {'key': 'โทรศัพท์', 'value': '038146124'},
 {'key': 'โทรสาร', 'value': '038146124'},
 {'key': 'ระดับที่เปิดสอน', 'value': 'อนุบาล-ประถมศึกษา'},
 {'key': 'วัน-เดือน-ปี ก่อตั้ง', 'value': '16 พฤษภาคม 2503'},
 {'key': 'อีเมล์', 'value': 'splt.school@gmail.com'},
 {'key': 'เว็บไซต์',
  'value': [{'text': 'เว็บไซต์โรงเรียน (สารสนเทศ)',
    'href': '../web/?School_ID=1020080005'}]},
 {'key': 'เครือข่ายพัฒนาคุณภาพการศึกษา', 'value': ''},
 {'key': 'องค์กรปกครองส่วนท้องถิ่น', 'value': 'หนองไม้แดง'},
 

In [8]:
for comment in soup.children: break
url = re.findall('url: (.*)\n', comment)[0]
parent_url = re.sub('[^/]*$', '', url)

In [9]:
image_file_image_re = r'(.*/|\?.*)'
re.sub(image_file_image_re,'','https://data.bopp-obec.info/emis/pic_school/1020080005.jpg?ivonb=42940')

'1020080005.jpg'

In [10]:
# test_image_file_name = ['1020080005_0.jpg',
# '1020080005_30.jpg',
# '1020080005_390.jpg',
# '1020080005_39.jpg',
# '1020080005_(10).jpg',
# '1020080005_(10).jpg',
# '1020080005_10.jpg',
# ]

# image_file_extention_re = r'\.[^\.]*$'
# [re.sub(image_file_extention_re, '', imname) for imname in test_image_file_name]

In [11]:
def download_image(image_url: str, image_file_dir: str):
  image = requests.get(image_url).content
  image_file_name = re.sub(image_file_image_re,'', image_url)
  image_file_path = image_file_dir+'/'+image_file_name
  if is_path_existed(image_file_path):
    return
  #   file_ext = re.findall(image_file_extention_re, image_file_path)
  #   print(file_ext)
  #   if not file_ext: return
  #   file_ext = file_ext[0]
  #   file_name = re.sub(image_file_extention_re, '', image_file_path)
  #   image_file_path = file_name+'_'+'0'+file_ext
  with open(image_file_path, 'wb') as im_file:
    im_file.write(image)

In [12]:
for div in soup.find_all('div'):
  if not div.find('div'):
    if 'ผู้อำนวยการโรงเรียน' in div.text:
      dir_name = 'principal'
    elif 'ตราสัญลักษณ์' in div.text:
      dir_name = 'logo'
    else:
      continue
    image_src = div.find('img').attrs['src']
    download_image(parent_url+image_src, '.')

In [13]:
soup = load_soup(sd['student'])

In [14]:
# all education year student data

# student_data_by_edu_year_urls = []
# for option in soup.find('select').find_all('option'):
#   if option.attrs['value']:
#     student_data_by_edu_year_urls\
#       .append(parent_url+option.attrs['value'])

In [15]:
# def student_table(soup):
#   col_headers = ['ชั้น/เพศ', 'ชาย', 'หญิง', 'รวม', 'ห้องเรียน']

#   for table in soup.find_all('table'):
#     if table.find('table'): continue
#     tab_col_headers = [td.text for td in table.find('tr').find_all('td')]
#     if any([col not in tab_col_headers for col in col_headers]): continue

#     table_row = []
#     for row in table.find_all('tr'):
#       cells = row.find_all('td')
#       if not table_row or (table_row and len(cells) == len(table_row[0])):
#         table_row.append([cell.text for cell in cells])
#     return table_row

In [16]:
pd.read_html(sd['student'])[-1]

,0,1,2,3,4
0,ชั้น/เพศ,ชาย,หญิง,รวม,ห้องเรียน
1,อบ.1,0,0,0,0
2,อบ.2,6,3,9,1
3,อบ.3,1,3,4,1
4,รวม อบ.,7,6,13,2
5,ป.1,4,2,6,1
6,ป.2,2,5,7,1
7,ป.3,5,4,9,1
8,ป.4,1,3,4,1
9,ป.5,3,4,7,1


In [54]:
staff_df = pd.read_html(sd['staff'])[-1]
staff_df.columns = staff_df.iloc[0]
# staff_df.set_index(['ตำแหน่ง',	'วิทยฐานะ',	'ระดับ'])
staff_df

,0,1,2,3,4,5
0,ตำแหน่ง,วิทยฐานะ,ระดับ,ข้าราชการครูและบุคลากรทางการศึกษา (คน),ข้าราชการครูและบุคลากรทางการศึกษา (คน),ข้าราชการครูและบุคลากรทางการศึกษา (คน)
1,ตำแหน่ง,วิทยฐานะ,ตำแหน่ง,ชาย,หญิง,รวม
2,1. ผู้อำนวยการโรงเรียน,-,คศ.1,0,0,0
3,1. ผู้อำนวยการโรงเรียน,ชำนาญการ,คศ.2,0,0,0
4,1. ผู้อำนวยการโรงเรียน,ชำนาญการพิเศษ,คศ.3,0,1,1
5,1. ผู้อำนวยการโรงเรียน,เชี่ยวชาญ,คศ.4,0,0,0
6,1. ผู้อำนวยการโรงเรียน,เชี่ยวชาญพิเศษ,คศ.5,0,0,0
7,1. ผู้อำนวยการโรงเรียน,รวม,-,0,1,1
8,2. รองผู้อำนวยการโรงเรียน,-,คศ.1,0,0,0
9,2. รองผู้อำนวยการโรงเรียน,ชำนาญการ,คศ.2,0,0,0


In [47]:
durable_goods_df = pd.read_html(sd['durable_goods'])[-2]
durable_goods_df.columns = durable_goods_df.iloc[0]
durable_goods_df  = durable_goods_df.iloc[1:, :]
durable_goods_df.set_index('ลำดับ', inplace=True)
durable_goods_df.iloc[:-1]

,รหัส,รายการ,รายการ(cm.),จำนวนที่ใช้ได้,จำนวนรอซ่อม,จำนวนรอจำหน่าย
ลำดับ,,,,,,
1,11001,ครุภัณฑ์การศึกษา,โต๊ะเก้าอี้นักเรียนก่อนประถมศึกษา,0,0,0
2,11002,ครุภัณฑ์การศึกษา,ชุดฝึกทักษะนักเรียนก่อนประถมศึกษา,0,0,0
3,11003,ครุภัณฑ์การศึกษา,ตู้เก็บอุปกรณ์นักเรียนก่อนประถมศึกษา,0,0,0
4,21001,ครุภัณฑ์สำนักงาน โรงเรียน,เครื่องถ่ายเอกสาร,1,0,0
5,21002,ครุภัณฑ์สำนักงาน โรงเรียน,เครื่องพิมพ์สำเนาระบบดิจิตอล,0,0,0
...,...,...,...,...,...,...
72,24008,ครุภัณฑ์งานบ้านและงานครัว,ถังน้ำขนาด 1000 ลิตร ขึ้นไป แบบไฟเบอร์กลาส,0,0,0
73,24009,ครุภัณฑ์งานบ้านและงานครัว,ถังน้ำขนาด 1000 ลิตร ขึ้นไป แบบพลาสติก,0,0,0
74,24010,ครุภัณฑ์งานบ้านและงานครัว,ถังน้ำขนาด 1000 ลิตร ขึ้นไป แบบสแตนเลส,0,0,0


In [97]:
b_soup = load_soup(sd['building'])
building_url = b_soup.find('iframe').attrs['src']
parent_url = re.sub('[^/]*$', '', building_url)
scrape_url(building_url, 'test.html');
soup = load_soup('test.html')

In [109]:
building_data = []
for cell in soup.find('table').find_all('td'):
  if not cell.find('img'):
    pair.update({
      'imaeg_description': [x.strip() for x in cell.text.strip().split('\n') if x.strip()]
    })
    building_data.append(pair)
  else:
    pair = {'image_url': parent_url+cell.find('img').attrs['src']}

In [110]:
building_data

[{'image_url': 'https://bobec.bopp-obec.info/b01/800054010625580120191011104209844453542.jpg',
  'imaeg_description': ['อาคารเรียน', 'สปช.105/29', 'ปีที่สร้าง 2558']},
 {'image_url': 'https://bobec.bopp-obec.info/b01/200100295014125542010120191011121748430521253.jpg',
  'imaeg_description': ['อาคารเรียน', 'พิเศษ 3 ชั้น', 'ปีที่สร้าง 2554']},
 {'image_url': 'https://bobec.bopp-obec.info/b02/800052021125570220191011120853335021609.jpg',
  'imaeg_description': ['อาคารอเนกประสงค์/โรงอาหาร/โรงฝึกงาน/หอประชุม',
   'อื่น อื่น(สร้างเอง)',
   'ปีที่สร้าง 2557']},
 {'image_url': 'https://bobec.bopp-obec.info/b02/200100293021825602010220191011122231328862316.jpg',
  'imaeg_description': ['อาคารอเนกประสงค์/โรงอาหาร/โรงฝึกงาน/หอประชุม',
   'พิเศษ',
   'ปีที่สร้าง 2560']},
 {'image_url': 'https://bobec.bopp-obec.info/b03/800051030625580320191011123214710103306.jpg',
  'imaeg_description': ['บ้านพักครู',
   'อื่น อื่น(สร้างเอง)',
   'ปีที่สร้าง 2558']},
 {'image_url': 'https://bobec.bopp-obec.info/b0